In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://rh-challenge-bucket.s3.amazonaws.com/amazon_reviews_us_Toys_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RDIJS7QYB6XNR|          5|            0|          0|   N|                Y|
|R36ED1U38IELG8|          5|            0|          0|   N|                Y|
| R1UE3RPRGCOLD|          2|            1|          1|   N|                Y|
|R298788GS6I901|          5|            0|          0|   N|                Y|
|  RNX4EXOBBPN5|          1|            1|          1|   N|                Y|
|R3BPETL222LMIM|          5|            0|          0|   N|                Y|
|R3SORMPJZO3F2J|          3|            2|          2|   N|                Y|
|R2RDOJQ0WBZCF6|          5|            0|          0|   N|                Y|
|R2B8VBEPB4YEZ7|          5|            0|          0|   N|                Y|
|R1CB783I7B0U52|          1|            0|          1|   N|     

In [6]:
vote_count = df.filter("total_votes>=20")

In [7]:
# Filter the new DataFrame and create a new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
filtered_df = vote_count.filter(vote_count["helpful_votes"]/vote_count["total_votes"]>=0.5)
filtered_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   11025729| ROP6ITXO8K5V2|B00LE9MZMW|     778942419|YJ Moyu 13x13x13 ...|            Toys|          5|           23|         27|   N|                Y|          Five Stars|Great cube, moves...| 2015-08-31|
|         US|   50123725|R3ND1LVU7AXCVF|B00LOV24VC|     363255349|Marshmallow Furni...|            Toys|          1|    

In [8]:
# Filter the DataFrame, and create a new DataFrame that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_filtered_df = filtered_df.filter(filtered_df["vine"] == "Y")
vine_filtered_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   53067113| R9I5FOLKU99RY|B00U1KA12G|     278300201|Qixels S1 Turbo D...|            Toys|          5|           19|         20|   Y|                N|11 Year Old Loves...|My daughter who i...| 2015-08-31|
|         US|   44759540|R3ED60RC69CCQ6|B00VJKTLMS|     917726757|Fisher-Price Nick...|            Toys|          5|    

In [9]:
# Filter the DataFrame, and create a new DataFramethat retrieves all the rows where a review was NOT written as part of the Vine program (paid), vine == 'N'.
not_vine_filtered_df = filtered_df.filter(filtered_df["vine"] == "N")
not_vine_filtered_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   11025729| ROP6ITXO8K5V2|B00LE9MZMW|     778942419|YJ Moyu 13x13x13 ...|            Toys|          5|           23|         27|   N|                Y|          Five Stars|Great cube, moves...| 2015-08-31|
|         US|   50123725|R3ND1LVU7AXCVF|B00LOV24VC|     363255349|Marshmallow Furni...|            Toys|          1|    

In [18]:
# Total reviews
total_reviews = filtered_df.count()
total_reviews

63294

In [19]:
# Total 5-star reviews
total_5_star = filtered_df.filter(filtered_df["star_rating"]==5).count()
total_5_star

30414

In [20]:
# Total 5-star Vine reviews
vine_count = vine_filtered_df.filter(vine_filtered_df["star_rating"]==5).count()
vine_count

432

In [21]:
# Total non-vine 5-star reviews
non_vine_count = not_vine_filtered_df.filter(not_vine_filtered_df["star_rating"]==5).count()
non_vine_count

29982

In [25]:
# Percentage of vine 5-star reviews
vine_percentage = vine_count/total_5_star *100
vine_percentage

1.4203985006904716

In [26]:
# Percentage of non-vine 5-star reviews
non_vine_percentage = non_vine_count/total_5_star*100
non_vine_percentage

98.57960149930953